In [1]:
!nvidia-smi

Wed Aug  9 11:13:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   32C    P8     6W / 180W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install torch
!pip install bitsandbytes
!pip install accelerate@git+https://github.com/huggingface/accelerate.git
!pip install transformers@git+https://github.com/huggingface/transformers.git
!pip install peft@git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 15.4 MB/s eta 0:00:0000:0100:01
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-98x53piu/accelerate_af2d6142ac664d70aee6f1449ea8713e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-98x53piu/accelerate_af2d6142ac664d70aee6f1449ea8713e
  Resolved https://github.com/huggingface/accelerate.git to commit b30a349078d5301411e8af421720eaa60e984fa2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.22.0.dev0-py3-none-any.whl size=250102 sha256=4fe3d4951f269aec41d31315dbeb31f5d2eb046cfa2af7c257fd1b978e80ea5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-91iranzi/wheels/60/53/1d/f8f7d9ed24f2b70cf9b37ecd31318a274049263effcc4b5bf3
Successfully built accelerate
  Cloning https://github.com/huggingface/tran

In [3]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [5]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 9.6 MB/s eta 0:00:00


In [6]:
base_model_name_or_path = "ybelkada/falcon-7b-sharded-bf16"

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, padding_side="right", trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [7]:
# Load the LoRA adapter

lora_model_name_or_path = "dominguesm/canarim-7b"
model = PeftModel.from_pretrained(model, lora_model_name_or_path)

model.eval()

if torch.__version__ >= "2":
    model = torch.compile(model)

In [8]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
"""

In [9]:
def instruct(
    instruction ,
    temperature=0.3,
    top_p=0.9,
    max_new_tokens=200,
    input=None,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    input_length = input_ids.shape[1]

    do_sample = temperature > 0.0
    
    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            max_new_tokens=max_new_tokens,
            **kwargs,
          ),
        return_dict_in_generate=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    splitted_generated_text = generated_text.split('\n')
    first_generated_text = splitted_generated_text[0]
    
    n = 1
    while first_generated_text == "":
        if splitted_generated_text[n]:
            first_generated_text = splitted_generated_text[n]
        else:
            break
    
    return first_generated_text

In [10]:
traducao = instruct("Translate the following text from english to portuguese.",input="He's never done anything.")

print(traducao)

/usr/local/lib/python3.9/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Ele nunca fez nada.


In [11]:
import pandas as pd

model_code = 'canarim7b'

try:
    opus_100_pt_validation = pd.read_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')
except:
    opus_100_pt_validation = pd.read_parquet('https://huggingface.co/api/datasets/opus100/parquet/en-pt/validation/0.parquet')

In [12]:
max_string = ""

for json in opus_100_pt_validation.translation:
    
    string = json['en'] 
    if len(string) > len(max_string):
        max_string = string
        
    string = json['pt'] 
    if len(string) > len(max_string):
        max_string = string
        
print("The biggest string is: " + max_string)

tokens = tokenizer.tokenize(max_string)
num_tokens = len(tokens)

print("The maximum tokens number is: " + str(num_tokens))

The biggest string is: having regard to the Annual Report of the Court of Auditors on the implementation of the budget concerning the financial year 2011, together with the institutions’ replies [3], and to the Court of Auditors’ special reports,having regard to the statement of assurance [4] as to the reliability of the accounts and the legality and regularity of the underlying transactions provided by the Court of Auditors for the financial year 2011 pursuant to Article 287 of the Treaty on the Functioning of the European Union,having regard to the Council’s recommendation of 12 February 2013 on discharge to be given to the Commission in respect of the implementation of the budget for the financial year 2011 (05752/2013 – C7-0038/2013),
The maximum tokens number is: 162


In [13]:
dataset = opus_100_pt_validation.translation

row_number = len(dataset)
current_row = 0

for json in opus_100_pt_validation.translation:
    current_row=current_row+1
    
    if model_code in json and json[model_code]:
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução existente: " + json[model_code] )
    else:
        originalString = json['en'] 
        traducao = instruct("Translate the following text from english to portuguese.",input=originalString)
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução: " + traducao)
        json[model_code]=traducao
        opus_100_pt_validation.to_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')
    

1/2000 - Tradução existente: Não faz nada.
2/2000 - Tradução existente: 50/50 shot?
3/2000 - Tradução existente: Yeah, and I'm letting you get your come-up. But you gonna be good?
4/2000 - Tradução existente: Que foi você, não foi?
5/2000 - Tradução existente: Then you go there!
6/2000 - Tradução existente: 'Cause you haven't washed it.
7/2000 - Tradução existente: Não, não pareceu.
8/2000 - Tradução existente: A: "O que você acha que aconteceu?"
9/2000 - Tradução existente: Se você me amasse, você me puxaria.
10/2000 - Tradução existente: - We have two suspects.
11/2000 - Tradução existente: - We opened the door.
12/2000 - Tradução existente: O empréstimo pode atrair um subsídio de juros para o qual se fazem contas no orçamento de operação.
13/2000 - Tradução existente: Não um centavo. Mikey Molloy fingiu ter um dos seus ataques... para que eu pudesse entrar quando ninguém estava olhando.
14/2000 - Tradução existente: I swear to you this challenge will be met
15/2000 - Tradução existe

In [14]:
opus_100_pt_validation.to_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')

In [16]:

saved_parquet = pd.read_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')
print(saved_parquet.translation[200])                        

{'canarim7b': 'Camargo Corrêa Group elaborated the Camargo Corrêa Business Code of Conduct, which gathers the principles guiding the relationship of companies, executives, shareholders, and employees.', 'en': 'To foster ethics in business, Camargo Corrêa Group elaborated the Camargo Corrêa Business Code of Conduct, which gathers the principles guiding the relationship of companies, executives, shareholders, and employees.', 'pt': 'Por incentivar a ética na condução dos negócios, o Grupo Camargo Corrêa elaborou o Código de Conduta Empresarial Camargo Corrêa, que reúne os princípios que norteiam o relacionamento de empresas, executivos, acionistas e profissionais.'}
